# Planejamento
Este documento descreve as etapas que serão seguidas para solução do teste técnico.


## Parte 1 - Análise exploratória
---
Esta etapa será fundamental para entender os dados e gerar insights sobre como estes podem ser utilizados para responder às perguntas do negócio, isto é, para criação dos sistemas das Partes 2 e 3.

Etapas:

- Importação dos pacotes e módulos utilizados
- Carregamento do arquivo csv criando um Dataframe
- Descrição dos dados
    - Análise do volume de dados
        - Será suficiente utilizar pandas e scikit-learn? Ou será necessário Dask ou Spark?
    - Análise dos tipos de dados
    - Tratamento de dados faltantes
        - Decidir melhor estratégia dependendo das características dos dados
    - Resumo geral dos dados
        - Estatística descritiva dos dados
            - Medidas de tendência central
                - Média, mediana            
            - Medidas de dispersão
                - Std, min, max, skew, kurtosis


- Feature engineering
    - Verificar se o tamanho da query é relevante para a tarefa de classificação

- Análise exploratória
    - Análise univariada - Qual a distribuição de cada variável?
        - Análise das classes utilizadas no sistema de classificação
        - Features numéricas
        - Features categóricas
     - Análise bivariada     
         - Qual a distribuição de produtos por categoria?
         - Como os preços variam por categoria?
         - Qual a variação dos atributos dos produtos (peso, preço, etc)? 
         - Quais os produtos mais vendidos?
         - Quais os produtos mais visualizados?
         - Qual a relação entre as vendas e visualizações dos produtos?
         - Qual o percentual de visualizações do produto que se convertem em vendas?
         - Como a média de vendas de produto varia por categoria?
         - Como a média de visualizações de produto varia por categoria?
         - Como a taxa de conversão do produto varia por categoria?
         - Como os pesos variam por categoria?
     - Análise multivariada - Há correlação entre as variáveis?
         - Variáveis numéricas         

## Parte 2 - Sistema de Classificação de Produtos
---

O objetivo desta etapa é criar um classificador de produtos utilizando o dataset fornecido e as transformações propostas, adotando as ideias geradas durante a análise exploratória.

## Método
- Separação dos dados em treinamento/teste
- Seleção de features utilizando feature importances 
- Criação de um pipeline para facilitar os experimentos com o classificador (além de permitir a aplicação dos mesmos passos ao conjunto de testes ao final do processo)
    - Processamento de linguagem natural
        - CountVectorizer
        - TfidfTransformer
    - Transformação da escala das features numéricas
        - MinMaxScaler (pois as features não apresentam distribuição normal)
- Serão experimentados 4 métodos que não assumem normalidade das features:   
    - SVC
    - Multinomial Naive Bayes    
    - KNeighborsClassifier
    - RandomForest

- A técnica de para geração dos conjuntos de treinamento e validação será a validação cruzada com Stratified K-Folds (sklearn.model_selection.StratifiedKFold), de forma a manter a mesma proporção entre as classes em cada par de conjuntos de treinamento e validação.
- Seleção do melhor utilizando a métrica precisão
    - Considerando que o impacto de falsos positivos é equivalente ao de falsos negativos
- Avaliação do melhor modelo utilizando o conjunto de testes
- Treinamento do modelo utilizando todos os dados disponíveis.
- Salvamento do modelo em arquivo para uso posterior.

# Parte 3 - Sistema de termos de busca
--- 

Como o classificador construído na etapa passada utiliza o termo de busca e o preço para predizer a categoria, iniciarei a investigação das classes de intenção buscando por faixas de preço, utilizando o algoritmo k-means.

Após a determinação do número de classes (grupos) através de testes variando o valor de K, os dados serão rotulados.

Como as faixas de preço não estarão disponíveis a partir do termo de busca, será criado um classificador que indicará o grupo a partir da query de busca.

O dataset será dividido em treinamento/validação e testes. O classificador de intenção será treinado a partir do conjunto de treinamento e seu desempenho será verificado utilizando o conjunto de teste.

## Parte 4 - Avaliação do Sistema de Classificação
---

- O sistema de classificação será avaliado nas etapas 2 e 3, através da separação do dataset em dados de treinamento, validação e teste.

## Parte 5 - Colaboração entre os sistemas
---

Nesta etapa os sistemas de classificação de categoria e intenção serão integrados. 

A partir de uma query, o sistema deve identificar a categoria e a classe de intenção do usuário.
Além disso, um sistema de recomendação irá indicar 10 produtos de acordo com a categoria e faixa de preço desejada pelo usuário.

Funcionamento do sistema final:

- Entrada
    - Termo de busca qualquer
- Saídas
    1. A categoria do termo de busca
    2. O nome da classe de intenção do usuário
    3. Uma lista com o id e o título dos dez produtos recomendados
    
A partir do termo de busca, será utilizado o classificador de intenções do usuário para identificar a classe de preços desejada. 

O termo de busca será utilizado em conjunto com a mediana dos preços da classe de intenção como entradas do classificador de categoria.

A partir da categoria e faixa de preços, serão selecionados os 10 produtos mais desejados do dataset, de acordo com a média de visualizações (feature 'view_count).